# PROYECTO ETL EN DATABRICKS - COSMETSAC
Autor: Brayan R. Neciosup Bolaños

Importante:
Como no se tiene desplegada una BD relacional en la nube, usaremos Unity Catalog 
y todas sus características para simular una BD relacional en Databricks. 

El Modelo Entidad Relación esta elaborado en MSSM, puedes visualizarlo en la imagen denominada: 
ModeloER-SQL.png 
en la carpeta database_loca O también puedes revisar el script: ScriptBDCosmetSAC.sql en la misma carpeta

### LIBRERÍAS UTILIZADAS

In [0]:
# Librerias a utilizar
from pyspark.sql import SparkSession # Puerta de acceso a todas las funcionalidades de apache spark
from pyspark.sql.functions import * # Funciones SQL
from pyspark.sql.types import * # Funciones de tipos de datos


### CONFIGURACIONES Y FUNCIONES UTILIZADAS PARA LA CARGA DE INFORMACIÓN EN UNITY CATALOG

In [0]:
#### Configuración de Unity Catalog para simular BD relacional
# A). Creación del Catálago (Nivel más alto en la jerarquía de Unity Catalog) 
# [Es como crear la BD en cualquier gestor de BD]
spark.sql("CREATE CATALOG IF NOT EXISTS cosmetsac")
print("Catálago creado exitosamente")

# B). Creación del Esquema (Segundo nivel en la jerarquía de Unity Catalog) 
# [Son como los esquemas dentro de cualquier gestor de BD] 
# [En caso no creamos conveniente crearlo, podemos usar el esquema "default"]
spark.sql("CREATE SCHEMA IF NOT EXISTS cosmetsac.ventas") # Permite administrar mejor cada entidad
print("Esquema ventas creado exitosamente")

# Importante: No crearemos los volumenes, porque las tablas donde se almacenarán los datos
#             serán entidades gobernada por Unity Catalog y permitirá usar el lenguaje SQL.

In [0]:
# Función encargada de cargar la información a la tabla de clientes 
def cargar_informacion_clientes_cosmetsac(dataframe_clientes):
    # dataframe_clientes.show()
    # Almacenaremos el dataframe en una delta table que será gobernado por Unity Catalog
    dataframe_clientes.write.format("delta").mode("overwrite").saveAsTable("cosmetsac.ventas.clientes")
    print("Datos de clientes registrados")
    # write: Modo de escritura
    # format("delta"): Es la forma nativa de Databricks para almacenar información (Optimizada y recomendada)
    # mode("overwrite"): Sobreescribe toda la información existente sin perder nada de datos.
    # saveAsTable("cosmetsac.ventas.clientes"): Permite guardarse como delta table optimizada

In [0]:
# Función encargada de cargar la información de tablas externas (Marcas,Categorias,FormasPagos)
# La similitud de estas 3 tablas, se debe a su estructura en sus campos.
def cargar_datos_tablas_externas(nombre_tabla,dataframe_tabla_externa):
    # dataframe_tabla_externa.show()
    # Almacenaremos el dataframe en una delta table que será gobernada por Unity Catalog
    jerarquia_unity_catalog_tabla = f"cosmetsac.ventas.{nombre_tabla}" # Formateamos la jerarquia en base al nombre de la tabla
    dataframe_tabla_externa.write.format("delta").mode("overwrite").saveAsTable(jerarquia_unity_catalog_tabla)
    print(f"Datos de {nombre_tabla} registrados")
    # write: Modo de escritura
    # format("delta"): Es la forma nativa de Databricks para almacenar información (Optimizada y recomendada)
    # mode("overwrite"): Sobreescribe toda la información existente sin perder nada de datos.
    # saveAsTable(jerarquia_unity_catalog_tabla): Permite guardarse como delta table optimizada

In [0]:
# Función encargada de cargar la información a la tabla productos
def cargar_datos_productos(dataframe_productos):
    # dataframe_productos.show()
    # Almacenaremos el dataframe en una delta table que será gorbernada por Unity Catalog
    dataframe_productos.write.format("delta").mode("overwrite").saveAsTable("cosmetsac.ventas.productos")
    print("Datos de productos registrados")
    # write: Modo de escritura
    # format("delta"): Es la forma nativa de Databricks para almacenar información (Optimizada y recomendada)
    # mode("overwrite"): Sobreescribe toda la información existente sin perder nada de datos.
    # saveAsTable("cosmetsac.ventas.productos"): Permite guardarse como delta table optimizada

In [0]:
# Funciión encargada de cargar la información a la tabla productos
def cargar_datos_pedidos(dataframe_pedidos):
    # dataframe_pedidos.show()
    # Almacenaremos el dataframe en una delta table que será gobernada por Unity Catalog
    dataframe_pedidos.write.format("delta").mode("overwrite").saveAsTable("cosmetsac.ventas.pedidos")
    print("Datos de pedidos registrados")
    # write: Modo de escritura
    # format("delta"): Es la forma nativa de Databricks para almacenar información (Optimizada y recomendada)
    # mode("overwrite"): Sobreescribe toda la información existente sin perder nada de datos.
    # saveAsTable("cosmetsac.ventas.productos"): Permite guardarse como delta table optimizada

In [0]:
# Funciión encargada de cargar la información a la tabla promociones
def cargar_datos_promociones(dataframe_promociones):
    # dataframe_pedidos.show()
    # Almacenaremos el dataframe en una delta table que será gobernada por Unity Catalog
    dataframe_promociones.write.format("delta").mode("overwrite").saveAsTable("cosmetsac.ventas.promociones")
    print("Datos de promociones registrados")
    # write: Modo de escritura
    # format("delta"): Es la forma nativa de Databricks para almacenar información (Optimizada y recomendada)
    # mode("overwrite"): Sobreescribe toda la información existente sin perder nada de datos.
    # saveAsTable("cosmetsac.ventas.productos"): Permite guardarse como delta table optimizada

In [0]:
# Función que permite traer los ID Compra por cada Cliente (ID Cliente)
def cargar_pedidos_ids_clientes(dataframe_clientes_ids):
    # Inicializamos un datafame con datos en 0, que se llenará dinamicamente más adelante.
    df_pedidos_clientes = spark.createDataFrame(schema=["PedidosClientesID","PedidosID"],data=[(0,0)]) 
    for i in dataframe_clientes_ids.select(col("PedidosClientesID")).collect(): # Recorremos cada ID de cliente
        # Retornamos un dataframe con los todos los pedidos (PedidosID) de cada cliente iterado del dataframe.
        df_ids_pedidos = spark.sql(f"SELECT PedidosID FROM cosmetsac.ventas.pedidos WHERE PedidosClientesID = {i[0]}")
        # print(f"CANTIDAD DE PEDIDOS DEL CLIENTE: {i[0]} es: {df_ids_pedidos.count()}") # Verificamos la cantidad de pedidos por cliente
        if df_ids_pedidos.count()>0: # Validamos que el dataframe anterior contenga información.
            # Creamos un diccionario que almacenará el ClienteID y sus respectivos PedidosID, esto permitirá crear un dataframe mas rápido
            diccionario_compras_clientes = {
                "PedidosClientesID": [i[0] for _ in range(1,df_ids_pedidos.count()+1)],
                "PedidosID": [i[0] for i in df_ids_pedidos.select(col("PedidosID")).collect()]
            }
            # print(diccionario_compras_clientes) # Verificamos la información del diccionario
            # Creamos un dataframe temporal que será llenado con la información del diccionario creado anteriormente.
            df_temp = spark.createDataFrame(data=list(zip(*diccionario_compras_clientes.values())),schema=["PedidosClientesID","PedidosID"])
            # .union() permite unir la información por filas entre dataframes, en este caso, 
            # el dataframe inicializado con datos en 0 y llenado dinamicamente con el dataframe temporal (df_temp)
            df_pedidos_clientes = df_pedidos_clientes.union(df_temp) # Llenado dinámico por cada dataframe del diccionario
        else:
            print("Dataframe sin datos")
    df_pedidos_clientes = df_pedidos_clientes.filter(
        (col("PedidosClientesID")!=0) & (col("PedidosID")!=0) 
        # Filtramos todos los datos diferentes de 0 del dataframe inicial que ha sido llenado dinámicamente
    )
    return df_pedidos_clientes # Retornamos el dataframe con los valores correctos


In [0]:
# Funciión encargada de preparar un dataframe con la estructura de la tabla detallepedidos.
def preparar_tabla_detalle_pedidos(df_clientes_pedidos_ids,df_ventas_f):
    df_detalle_pedidos = df_ventas_f.select( # Seleccionamos las columnas necesarias del dataframe de ventas
        col("ID Cliente").alias("PedidosClientesID"), # Renombramos la columna ID Cliente a PedidosClientesID para facilitar el .join()
        col("ID Producto"),col("Cantidad"),col("Precio Unitario (S/)"),col("Promocion ID")
    )
    # Mediante .join() realizamos la unión de los dataframes (df_clientes_pedidos_ids y df_ventas_f) de manera columnar.
    df_detalle_pedidos = df_detalle_pedidos.join(df_clientes_pedidos_ids,how="inner",on="PedidosClientesID")
    return df_detalle_pedidos.sort(col("PedidosID").asc()) # Retornamos el dataframe df_detalle_pedidos ordenados por PedidosID (ascendente)

In [0]:
# Función encargada de cargar la información a la tabla de detalle pedidos
def cargar_datos_detalle_pedidos(dataframe):
    # Preparamos dataframe para una correcta coherencia con la estructura definida de la tabla:
    dataframe = dataframe.select(
        col("PedidosID").alias("DetallePedidosPedidoID"),
        col("ID Producto").alias("DetallePedidosProductoID"),
        col("Precio Unitario (S/)").alias("DetallePedidosPrecioVenta"),
        col("Cantidad").alias("DetallePedidosCantidad"),
        col("Promocion ID").alias("DetallePedidosPromocionID")
    )
    # dataframe.show()
    # Almacenaremos el dataframe en una delta table que será gobernada por Unity Catalog
    dataframe.write.format("delta").mode("overwrite").saveAsTable("cosmetsac.ventas.detallepedidos")
    print("Datos de Detalle Pedidos registrados")
    # write: Modo de escritura
    # format("delta"): Es la forma nativa de Databricks para almacenar información (Optimizada y recomendada)
    # mode("overwrite"): Sobreescribe toda la información existente sin perder nada de datos.
    # saveAsTable("cosmetsac.ventas.productos"): Permite guardarse como delta table optimizada



### PROCESO ETL (EXTRAER - TRANSFORMAR Y CARGAR)

#### INCONSISTENCIAS POR SUBSANAR EN LOS ARCHIVOS

Estas son las inconsistencias encontradas en los archivos : 
CLIENTES-EMPRESA-COSMETSAC.csv | PRODUCTOS-EMPRESA-COSMETSAC.csv | VENTAS-EMPRESA-COSMETSAC.csv

Archivo CLIENTES-EMPRESA-COSMETSAC.csv: 

- Los nombres y apellidos de los clientes presentan guiones(-) entre sus registros.
- Los nombres y apellidos de los clientes se encuentran en una sola columna.
- El teléfono de los clientes presentan el prefijo "51".
- Los clientes no tienen un ID asignado.

ARCHIVO PRODUCTOS-EMPRESA-COSMETSAC.csv:

- Las categorias y marcas no tienen un ID asignado.
- Los ID de cada producto tienen un prefijo "P" y tienen ceros inicialmente.
   (Se omite esta columna en el ETL y se asigna automáticamente)

ARCHIVO VENTAS-EMPRESA-COSMETSAC.csv:

- Las ventas registradas no tiene un ID asignado (Incoherencia fuerte).
- El ID de cada cliente tiene un prefijo de C junto con ceros inicialmente.
- Redudancia de información con columnas que deberían estar presentes en otras tablas.
- La columna Promoción tiene sus campos propios, pero no un ID asignado correctamente.
- La columna PagosDescripcion tiene su campo propio, pero no un ID asignado correctamente.

 SOLUCIÓN: CREAR UNA BD RELACIONAL SIMULADA EN DATABRICKS UTILIZANDO UNITY CATALOG



#### CÓDIGO ETL - EMPRESA COSMET S.A.C

In [0]:
# Archivo CLIENTES-EMPRESA-COSMETSAC.xlsx cargada previamente a Unity Catalog en formato delta table(clientes_empresa_cosmetsac)

#### EXTRAER
clientes_cosmetsac = spark.sql("SELECT * FROM workspace.exercises.clientes_empresa_cosmetsac")
# clientes_cosmetsac.show() # Leemos las 5 primeras filas de la tabla

#### Transformar
#-- Limpiar la columna CLIENTE de guiones que existen
clientes_cosmetsac = clientes_cosmetsac.withColumns({
    "CLIENTE": regexp_replace(col("CLIENTE"),r'-',' '), # Función que permite reemplazar (-) por espacios en blanco.
    "TELEFONO": cast(StringType(),col("TELEFONO")) # Convertimos a String el Telefono para poder extraer el prefijo 51
})
clientes_cosmetsac = clientes_cosmetsac.withColumns({
    "CLIENTE":upper(col("CLIENTE")), # La información de los nombres y apellidos del clientes, lo convertimos a mayúsculas
    "TELEFONO":cast(IntegerType(),regexp_replace(col("TELEFONO"),r'^51','')) 
    # Reemplazamos el prefijo "51" por un espacio en blanco y convertimos a tipo IntegerType() para manetener la integridad de datos.
})
clientes_cosmetsac = clientes_cosmetsac.withColumns({
    "Primer Nombre Cliente":split(col("CLIENTE")," ").getItem(0), # .split() Permite separar la información
    "Segundo Nombre Cliente":split(col("CLIENTE")," ").getItem(1),# basandose en un delimitador, para poder convertirlo a un array
    "Primer Apellido Cliente":split(col("CLIENTE")," ").getItem(2), # mismo array, al cu+al accedemos a cada elemtno respectivamente
    "Segundo Apellido Cliente":split(col("CLIENTE")," ").getItem(3) # con .getItem(IndiceElemento)
})

clientes_cosmetsac = clientes_cosmetsac.withColumns({
    "Apellidos":concat_ws(', ',col("Primer Apellido Cliente"),col("Segundo Apellido Cliente")), # Concatemos el primer y segundo apellido
    "Nombres":concat_ws(', ',col("Primer Nombre Cliente"),col("Segundo Nombre Cliente")) # Concatemos el primer y segundo nombre
    # Ambas concatenaciones las volvemos en una sola columna respectivamente
})

clientes_cosmetsac = clientes_cosmetsac.select(
    col("Apellidos"),col("Nombres"),col("DNI"),col("CORREO ELECTRONICO"),col("TELEFONO") 
    # Seleccionamos en orden las columnas de acuerdo a la estructura de la tabla clientes
)

# clientes_cosmetsac.show()
# Preparamos dataframe para una correcta coherencia con la estructura definida de la tabla:
clientes_bd = clientes_cosmetsac.withColumn(
    "ClientesID",                  # Agrego una columna para el ID del cliente
    monotonically_increasing_id()+1 # Columna con datos autoincrementables empezando en 1
)
clientes_bd = clientes_bd.select( # Modificamos el nombre de ciertas columnas en base a la tabla clientes
    col("ClientesID"),col("Apellidos").alias("ClientesApellidos"),
    col("Nombres").alias("ClientesNombres"),col("DNI").alias("ClientesDNI"),
    col("CORREO ELECTRONICO").alias("ClientesCorreoElectronico"),col("TELEFONO").alias("ClientesTelefono")
)
# clientes_bd.show()

#### CARGAR
# FUNCIÓN PARA CONVERTIR ESTE DATAFRAME A UN DELTA TABLE EN UNITY CATALOG
cargar_informacion_clientes_cosmetsac(clientes_bd)

In [0]:
# Archivo PRODUCTOS-EMPRESA-COSMETSAC.xlsx cargada previamente a Unity Catalog en formato delta table(productos_empresa_cosmetsac)

#### EXTRAER

productos_cosmetsac = spark.sql("SELECT * FROM workspace.exercises.productos_empresa_cosmetsac")
# productos_cosmetsac.show() 

#### TRANSFORMAR

productos_cosmetsac = productos_cosmetsac.select( # Seleccionamos las columnas específicas en base a la estructura de la tabla Productos
    col("Producto"),col("Precio regular (S/)"),col("Stock Actualizado"),
    col("Marca"),col("Categoría")
)
productos_cosmetsac = productos_cosmetsac.withColumnsRenamed({
    "Precio regular (S/)":"Precio Compra", # Renombramos la columna Precio Regular (S/) a Precio Compra
    "Stock Actualizado":"Stock" # Renombramos la columna Stock Actualizado a Stock
})

# EXTRAEMOS LAS MARCAS
marcas_unicas = productos_cosmetsac.select(col("Marca")).dropDuplicates() # Eliminamos duplicados
diccionario_marcas = {
    # Permite crear un ID por cada marca única existente
    "Marca ID":[i for i in range(1,marcas_unicas.count()+1)],
    # Retorna cada marca única existente.
    "Marca":[i[0] for i in marcas_unicas.select(col("Marca")).collect()]
}
df_marcas = spark.createDataFrame(list(zip(*diccionario_marcas.values())),["Marca ID","Marca"])
# Preparamos dataframe para una correcta coherencia con la estructura definida de la tabla:
marcas_bd = df_marcas.select(col("Marca ID").alias("MarcasID"),col("Marca").alias("MarcasDescripcion"))
# df_marcas.show()

# EXTRAEMOS LAS CATEGORIAS

categorias_unicas = productos_cosmetsac.select(col("Categoría")).dropDuplicates() # Eliminamos duplicados
diccionario_categorias = {
    # Permite crear un ID por cada categoría única existente
    "Categoria ID":[i for i in range(1,categorias_unicas.count()+1)],
    # Retorna cada categoría única existente
    "Categoría": [i[0] for i in categorias_unicas.select(col("Categoría")).collect()]
}
df_categorias = spark.createDataFrame(list(zip(*diccionario_categorias.values())),["Categoria ID","Categoría"])
# Preparamos dataframe para una correcta coherencia con la estructura definida de la tabla:
categorias_bd = df_categorias.select(col("Categoria ID").alias("CategoriasID"),col("Categoría").alias("CategoriasDescripcion"))
# df_categorias.show()

# Accedemos a los dataframes anteriores y con .join() unificamos la información a nivel de columna
# Es como realizar un INNER JOIN de 2 tablas en SQL SERVER, permitiendo tener el ID de la Marca y Categoria
# respectivamente donde aparece en la información.
productos_cosmetsac = productos_cosmetsac.join(df_marcas,"Marca")
productos_cosmetsac = productos_cosmetsac.join(df_categorias,"Categoría")

# Seleccionamos columnas necesarias.
productos_cosmetsac = productos_cosmetsac.select(
    col("Producto"),col("Precio Compra"),col("Stock"),
    col("Marca ID"),col("Categoria ID")
)
# Preparamos dataframe para una correcta coherencia con la estructura definida de la tabla:
productos_bd = productos_cosmetsac.withColumn(
    "ProductosID",                  # # Agrego una columna para el ID del producto
    monotonically_increasing_id()+1 # Columna con datos autoincrementables empezando en 1
)
productos_bd = productos_bd.select(
    col("ProductosID"),col("Producto").alias("ProductosDescripcion"),
    col("Precio Compra").alias("ProductosPrecioCompra"),col("Stock").alias("ProductosStock"),
    col("Marca ID").alias("ProductosMarcasID"),col("Categoria ID").alias("ProductosCategoriasID")
)
# productos_bd.show()

#### CARGAR

# Funciones para cargar información ....
cargar_datos_tablas_externas(nombre_tabla="marcas",dataframe_tabla_externa=marcas_bd) # Tabla marcas
cargar_datos_tablas_externas(nombre_tabla="categorias",dataframe_tabla_externa=categorias_bd) # Tabla categorias
cargar_datos_productos(productos_bd)


In [0]:
# Archivo VENTAS-EMPRESA-COSMETSAC.xlsx cargada previamente a Unity Catalog en formato delta table(ventas_empresa_cosmetsac)

#### EXTRAER

ventas_cosmetsac = spark.sql("SELECT * FROM workspace.exercises.ventas_empresa_cosmetsac")
# ventas_cosmetsac.show()

#### TRANSFORMAR

# # Extraemos las formas de pago
formas_pago_unicas = ventas_cosmetsac.select(col("PagosDescripcion")).dropDuplicates()
diccionario_formas_pago = {
    # Permite crear un ID por cada forma de pago única existente
    "FormaPagoID":[i for i in range(1,formas_pago_unicas.count()+1)],
    # Retorna cada descripción de las formas de pago únicas existentes
    "PagosDescripcion":[i[0] for i in formas_pago_unicas.select(col("PagosDescripcion")).collect()]
}
df_formas_pago = spark.createDataFrame(data=list(zip(*diccionario_formas_pago.values())),schema=["FormaPagoID","PagosDescripcion"])
# df_formas_pago.show()

# Extraemos las promociones
promociones_unicas = ventas_cosmetsac.select(
    col("Promocion"),col("FechaInicio"),col("FechaFin")
).dropDuplicates()

# Agregamos columna Descuento de la descripción
promociones_unicas = promociones_unicas.withColumns({
    "Promocion ID":                   # Agregamos una columna para el Promocion ID
    monotonically_increasing_id()+1,  # Columna con datos autoincrementables empezando en 1
    "Descuento":
    (regexp_extract(col("Promocion"),r'\d+',0)/100),
    "Estado":
    lit(0)
})
# promociones_unicas.show()
# Preparamos dataframe para una correcta coherencia con la estructura definida de la tabla:
promociones_bd = promociones_unicas.select(
    col("Promocion ID").alias("PromocionesID"),
    col("Promocion").alias("PromocionesDescripcion"),
    col("Descuento").alias("PromocionesDescuento"),
    col("FechaInicio").alias("PromocionesFechaInicio"),
    col("FechaFin").alias("PromocionesFechaFin"),
    col("Estado").alias("PromocionesEstado")
)
promociones_bd = promociones_bd.withColumn(
    "PromocionesEstado",
    when(
        # Mantendremos siempre activa la promoción con 0% (Regla de negocio)
        col("PromocionesDescripcion")=='Sin Promoción: 0% de Descuento',lit(1) 
    ).otherwise(lit(0))
)
# promociones_bd.show()

# Construimos el dataframe ventas final para la fase de Carga
# 1️⃣ Unimos con el dataframe de Formas Pagos
df_ventas_cosmetsac = ventas_cosmetsac.join(df_formas_pago,on="PagosDescripcion",how="inner")
# 2️⃣ Unimos con el dataframe de Promociones
df_ventas_cosmetsac = df_ventas_cosmetsac.join(promociones_unicas,how="inner",on="Promocion")
# 3️⃣ Estandarizamos el ID Cliente e ID Producto
df_ventas_cosmetsac = df_ventas_cosmetsac.withColumns({
    "ID Cliente":
    regexp_replace(col("ID Cliente"),r'^C','').cast(dataType=IntegerType()),
    "ID Producto":
    regexp_replace(col("ID Producto"),r'^P','').cast(dataType=IntegerType())
})
# df_ventas_cosmetsac.show() 

# Seleccionamos columnas necesarias para poblar la tabla Pedidos en la fase de Carga
df_pedidos = df_ventas_cosmetsac.select(col("FechaCompra"),col("ID Cliente"),col("FormaPagoID")).sort(col("FechaCompra").asc())
# df_pedidos.show()

# Preparamos dataframe para una correcta coherencia con la estructura definida de la tabla:
formas_pagos_bd = df_formas_pago.select(
    col("FormaPagoID").alias("FormasPagosID"),
    col("PagosDescripcion").alias("FormasPagosDescripcion")
)
# formas_pagos_bd.show()
pedidos_bd = df_pedidos.withColumn(
    "PedidosID",                    # Agrego una columna para el ID de pedidos
    monotonically_increasing_id()+1 # Columna con datos autoincrementables empezando en 1
)
pedidos_bd = pedidos_bd.select(
    col("PedidosID"),col("FechaCompra").alias("PedidosFechaRegistro"),
    col("ID Cliente").alias("PedidosClientesID"),col("FormaPagoID").alias("PedidosFormasPagosID")
)
# pedidos_bd.show()

### CARGAR
# Funciones para cargar las tablas formas de pago, promociones y pedidos
cargar_datos_promociones(dataframe_promociones=promociones_bd)
cargar_datos_tablas_externas(nombre_tabla="formaspagos",dataframe_tabla_externa=formas_pagos_bd)
cargar_datos_pedidos(dataframe_pedidos=pedidos_bd)

In [0]:
# Archivo VENTAS-EMPRESA-COSMETSAC.xlsx cargada previamente a Unity Catalog en formato delta table(ventas_empresa_cosmetsac)
# ETL ➡️ Tabla DetallePedidos

#### EXTRAER Y TRANSFORMAR
# Extraer los IDs de los clientes que tienen un pedido registrado

clientes_ids = pedidos_bd.select(col("PedidosClientesID")).dropDuplicates().sort(col("PedidosClientesID").asc())
# clientes_ids = clientes_ids.filter((col("PedidosClientesID")==1) | (col("PedidosClientesID")==2) | (col("PedidosClientesID")==3))
# clientes_ids.show()

#### CARGAR
# Función que permite extraer todos los IDs de Pedidos de los clientes
clientes_ids_pedidos = cargar_pedidos_ids_clientes(dataframe_clientes_ids=clientes_ids)
# Función que prepara el dataframe de la función anterior, para el poblado de la tabla DetallePedidos
df_detalle_pedidos = preparar_tabla_detalle_pedidos(df_clientes_pedidos_ids=clientes_ids_pedidos,df_ventas_f=df_ventas_cosmetsac)
# Función que permite realizar la carga de información a la tabla DetallePedidos
cargar_datos_detalle_pedidos(df_detalle_pedidos) 



